In [ ]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

HEADER = 'headless'
PAGE_LOADING_TIME = 10
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

options = webdriver.ChromeOptions()
options.add_argument(HEADER)
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
options.add_argument("lang=ko_KR")

chromedriver = '/usr/local/chromedriver/chromedriver'
driver = webdriver.Chrome(service=Service(chromedriver), options=options)

driver.get('https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000157294&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small', )

# open review page
WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#reviewInfo > a"))).click()

names, dates, reviews, stars = [], [], [], []
clicked_next = False
next_page_number = 2

while True:
    elems_page = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.pageing > a")))
    pages = []

    for elem in elems_page:
        pages.append(elem.get_attribute("data-page-no"))
    pages = list(map(int, pages))

    # if next button exists
    if pages[-1]%10 == 1:
        for page in pages:
            print("current page :", page)

            # find reviews of current page
            elems_review = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            number_of_review = len(elems_review)
            print("count the review :", number_of_review)
            
            for i in range(number_of_review):
                print("%dth review :"%i)
                name = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                name = name.text
                date = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                time.sleep(PAGE_LOADING_TIME)
                date = date.text
                review = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))
                review = review.text
                star = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.review_point > span"%(i+1)))).get_attribute("style")
                star = re.sub(r'[^0-9]', '', star)
                star = str(int(int(star) / 20))

                names.append(name)
                dates.append(date)
                reviews.append(review)
                stars.append(star)
            
            if page != pages[-1]:
                if clicked_next == True:
                    print("clicked_next True, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(next_page_number))))))
                    next_page_number += 1
                    
                elif clicked_next == False:
                    print("clicked next False, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(page))))))
                    
            elif page == pages[-1]:
                print("next page arrow click (occur document navigation)")
                driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a.next")))))
                clicked_next = True
                next_page_number = 3
                break
            
    elif pages[-1]%10 != 1:
        print("last page")
        for page in pages:
            print("current page :", page)
            
            # find reviews of current page
            elems_review = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            print("count the review :", number_of_review)
            
            for i in range(number_of_review):
                print("%dth review :"%i)
                name = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                name = name.text
                date = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                
                date = date.text
                review = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))
                review = review.text
                star = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.review_point > span"%(i+1)))).get_attribute("style")
                star = re.sub(r'[^0-9]', '', star)
                star = str(int(int(star) / 20))

                names.append(name)
                dates.append(date)
                reviews.append(review)
                stars.append(star)
        
            if page != pages[-1]:
                if clicked_next == True:
                    print("clicked_next True, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(next_page_number))))))
                    next_page_number += 1
        
        if pages[-1] == page:
            break

result_dict = []
for i in range(len(names)):
    result_dict.append({
        'NAME' : names[i],
        'DATE' : dates[i],
        'REVIEW' : reviews[i],
        'STAR' : stars[i]
    })
result_df = pd.DataFrame(result_dict)
            
driver.quit()